In [1]:
%load_ext autoreload
from config import Marco,content_generator
import pandas as pd
import os 
from utils import get_selected_cols_df,get_selected_values_df,joinStr,ensure_dir,get_daily_report_df,getDate_in_month,get_selected_values_over_df,greeting,get_night_data
from datetime import datetime 
from glob import glob
import warnings
warnings.filterwarnings("ignore")


In [6]:
data_path = 'misc'
data_file = "US_SHIPMENT_MONITORING_EVERY2"
data_file1 = "IOD_MONITORING_DETAIL_US_"
main_data_path =sorted(glob(f".\\{data_path}\\{data_file}*.csv"))[-1]

##########

df = pd.read_csv(main_data_path,encoding= 'unicode_escape')
print(main_data_path)
str(datetime.now())[0:10]

.\misc\US_SHIPMENT_MONITORING_EVERY2-2020-09-16.csv


'2020-09-16'

In [8]:
df.columns

# pull up HQ/Division/Prod1/Customer/LD#/Value and latest ETA fields

Index(['THIS_MONTH_SALES', 'ESTIMATED_TIME_ARRIVAL', 'BUYERPO',
       'GERP_ORDER_NO', 'GERP_PICK_NUMBER', 'TMS_SHIPMENT_NUMBER',
       'SHIPMENT_STATUS', 'LOAD_ID', 'LOAD_STATUS', 'WMS_STATUS', 'MODEL',
       'MOVE_ORDER_LINE_ID', 'SOURCE_LINE_ID', 'DIVISION_CODE', 'ITEM_QTY',
       'TOT_WGT', 'TOT_VOL', 'ORDER_VALUE', 'CUST_CD', 'CUST_NAME',
       'FRM_SHPG_LOC_CD', 'TO_SHPG_LOC_CD', 'TO_CTY_NAME', 'TO_STA_CD',
       'TO_PSTL', 'CARR_CD', 'SRVC_CD', 'ORDER_CREATION_DATE', 'DROP_IN_TMS',
       'LD_CRTD_DTT', 'WMS_RELEASE_DATE', 'SHPD_DTT', 'ORI_DELY_FROM',
       'ORI_DELY_TO', 'FRM_DLVY_DTT', 'TO_DLVY_DTT', 'PICK_APT_FRM',
       'PICK_APT_TO', 'APT_REQUEST_DATE', 'APT_LG_SUGGEST_DATE',
       'DROP_APT_FRM', 'DROP_APT_TO', 'APT_NO', 'PRODUCT_LEVEL1_CODE',
       'PRODUCT_LEVEL2_CODE', 'PRODUCT_LEVEL3_CODE', 'PRODUCT_LEVEL4_CODE',
       'WAVEKEY', 'REMARK', 'LDD_DIST', 'FREIGHTTERMSENUMVAL',
       'PICKUPFROMDATETIME', 'PRO_NO', 'SHIP_YN', 'FRM_PSTL_CD', 'HQ',
       'LAST_S

In [14]:
selected_df = df[['CUST_NAME','PRODUCT_LEVEL1_CODE','DIVISION_CODE','LOAD_ID','ORDER_VALUE','ETA1', 'ETA2', 'ETA3', 'ETA4','TO_STA_CD']]

In [15]:
data_file1 = "IOD_MONITORING_DETAIL_US_"
main_data_path =sorted(glob(f".\\{data_path}\\{data_file1}*.csv"))[-1]
df1 = pd.read_csv(main_data_path,encoding= 'unicode_escape')
area_dict = dict(zip(df1['Load ID'], df1['To Address']))
STORM_STATE = ['AL','GA','SC','NC']
STORM_STATE_df = selected_df[selected_df['TO_STA_CD'].isin(STORM_STATE)]

In [47]:
def get_city(text):
    return text.split(",")[0].split()[-1]

def get_zip(text):
    return text.split(",")[1].split()[1]

In [41]:

STORM_STATE_df['to_address'] = STORM_STATE_df.LOAD_ID.apply(lambda x : area_dict[int(x)] if x is not pd.NA and area_dict.get(x)  else "no address")

In [53]:
STORM_STATE_df['County or City'] = STORM_STATE_df['to_address'].apply(lambda x : get_city(x) if x != "no address" else "no city and county details")

In [54]:
STORM_STATE_df['zipcode'] = STORM_STATE_df['to_address'].apply(lambda x : get_zip(x) if x != "no address" else "no zipcode details")

In [55]:
STORM_STATE_df.to_csv("storm_sally_area_details.csv",index=False)

In [61]:
writer = pd.ExcelWriter('storm_sally_area_details.xlsx', engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet("US_SHIPMENT_MONITORING_EVERY2-2")
writer.sheets["US_SHIPMENT_MONITORING_EVERY2-2"] = worksheet
df.to_excel(writer,sheet_name="US_SHIPMENT_MONITORING_EVERY2-2",startrow=0 , startcol=0,index= False)

In [62]:
worksheet=workbook.add_worksheet("result_sally_storm")
writer.sheets["result_sally_storm"] = worksheet
STORM_STATE_df.to_excel(writer,sheet_name="result_sally_storm",startrow=0 , startcol=0,index= False)

In [63]:
writer.save()